In [17]:
# Required libs
!pip3 install svgpathtools opencv-python numpy aggdraw

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 993 kB 1.1 MB/s            


In [10]:
def to_tuple(compl):
    return (int(compl.real * 3), int(compl.imag * 3))

In [12]:
import math

def calc_len(vector_1, vector_2):
    unit_vector_1 = vector_1 / np.linalg.norm(vector_1)
    unit_vector_2 = vector_2 / np.linalg.norm(vector_2)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    return np.arccos(dot_product) / math.pi * 360

In [13]:
def two_points_dist(a, b):
    return (abs(a[0] - b[0]) ** 2 + abs(a[1] - b[1]) ** 2) ** 0.5

In [14]:
def dfs_on_segments(v, color, segm, usd, gr):
    usd[v] = True
    gr[v] = color
    
    curr_vector = [segm[v][1][0] - segm[v][0][0], segm[v][1][1] - segm[v][0][1]]
    
    for u, _ in enumerate(segm):
        vector_to = [segm[u][1][0] - segm[u][0][0], segm[u][1][1] - segm[u][0][1]]
        vector_to_rev = [segm[u][0][0] - segm[u][1][0], segm[u][0][1] - segm[u][1][1]]
        
        min_angle = min(calc_angle(curr_vector, vector_to), calc_angle(curr_vector, vector_to_rev))

        minn = two_points_dist(segm[v][1], segm[u][1])
        if not usd[u]:
            minn = min(minn, two_points_dist(segm[v][1], segm[u][1]))
            minn = min(minn, two_points_dist(segm[v][1], segm[u][0]))
            minn = min(minn, two_points_dist(segm[v][0], segm[u][1]))
            # minn += 1  # Eliminate 0 (due to possible division by zero
            
            dfs_on_segments(u, color, segm, usd, gr)

In [75]:
import cv2
import numpy as np
from random import randint

from PIL import Image
import aggdraw

from svgpathtools import svg2paths, wsvg, svg2paths2


class Point:
    def __init__(self, i, j):
        self.real = i
        self.imag = j
        
        # TODO: use only x and y
        self.x = i
        self.y = j

    def __sub__(self, other):
        return Point(self.x - other.x, self.y - other.y)

    def length(self):
        return (self.x * self.x + self.y * self.y) # Not taking square root for optimisation purposes



ANGLE_EPS = 6.28 / 5
MAX_STEEPNESS = 10
IMAGE_SIZE_MULTIPLIER = 1.7

block_size = 366

segments = [[[] for i in range(6)] for j in range(6)]

paths, attributes, svg_attributes = svg2paths2(f'svg/{5}.svg')


img = np.zeros((6, 6, 1000, 1000, 3))


for _, path in enumerate(paths):
    for curve in path:  # Path contains all curves in image        
        i, j = int(curve.start.real // block_size), int(curve.start.imag // block_size)
        p0 = Point(curve.start.real - block_size * i, curve.start.imag - block_size * j)
        p3 = Point(curve.end.real - block_size * i, curve.end.imag - block_size * j)
        
        if (type(curve).__name__ == "CubicBezier"):
            # Two more points for curve
            p1 = Point(curve.control1.real - block_size * i, curve.control1.imag - block_size * j)
            p2 = Point(curve.control2.real - block_size * i, curve.control2.imag - block_size * j)

            vector_1 = [p1.real - p0.real, p1.imag - p0.imag]
            vector_2 = [p3.real - p2.real, p3.imag - p2.imag]

            img[i, j] = cv2.line(img[i, j], to_tuple(p0), to_tuple(p1), (255, 0, 0), 1)
            img[i, j] = cv2.line(img[i, j], to_tuple(p2), to_tuple(p3), (0, 255, 0), 1)
            
            chord = (p3 - p0).length()
            cont_net = (p0 - p1).length() + (p2 - p1).length() + (p3 - p2).length()
            arc_length = (cont_net + chord) / 2;  # This is just an approximation
            
            steepness = arc_length / calc_len(vector_1, vector_2)
            
            if (steepness > MAX_STEEPNESS):
                segments[i][j].append([p0, p1, p2, p3])

        elif (type(curve).__name__ == "Line"):
            img[i, j] = cv2.line(img[i, j], to_tuple(p0), to_tuple(p3), (255, 0, 0), 1)
            segments[i][j].append([p0, p0, p3, p3])
    

    for i in range(6):
        for j in range(6):
            name = str(j * 6 + i).zfill(2)
        
            # Setup
            curves = Image.new("RGBA", (500, 500)) # last part is image dimensions
            draw = aggdraw.Draw(curves)
            fill = aggdraw.Brush("transparent")
            
            for ind, segm in enumerate(segments[i][j]):
                outline = aggdraw.Pen((randint(0, 255), randint(0, 255), randint(0, 255)), 4)
                pathstring = f"m{segm[0].x * IMAGE_SIZE_MULTIPLIER} {segm[0].y * IMAGE_SIZE_MULTIPLIER} C{segm[1].x * IMAGE_SIZE_MULTIPLIER} {segm[1].y * IMAGE_SIZE_MULTIPLIER},{segm[2].x * IMAGE_SIZE_MULTIPLIER} {segm[2].y * IMAGE_SIZE_MULTIPLIER},{segm[3].x * IMAGE_SIZE_MULTIPLIER} {segm[3].y * IMAGE_SIZE_MULTIPLIER}"
                
                symbol = aggdraw.Symbol(pathstring)
                draw.symbol((0, 0), symbol, outline)

#                 draw.line((segm[0].x, segm[0].y, segm[3].x, segm[3].y), outline)  # Debug line
                draw.flush()
            
            smb_num = 5 * 36 + j * 6 + i
            print(smb_num)
            print(f'res/{smb_num // 48 + 1}/{smb_num % 48}_th.png')
            
            org = cv2.imread(f'res/{smb_num // 48 + 1}/{smb_num % 48}_th.png')
            cv2.imwrite(f"curves/curves{name}_org.png", org)
            
            curves.save(f"curves/curves{name}.png")

/tmp/ipykernel_19368/2908803390.py:79: RuntimeWarning: divide by zero encountered in double_scalars
  steepness = arc_length / calc_len(vector_1, vector_2)


180
res/4/36_th.png
186
res/4/42_th.png
192
res/5/0_th.png
198
res/5/6_th.png
204
res/5/12_th.png
210
res/5/18_th.png
181
res/4/37_th.png
187
res/4/43_th.png
193
res/5/1_th.png
199
res/5/7_th.png
205
res/5/13_th.png
211
res/5/19_th.png
182
res/4/38_th.png
188
res/4/44_th.png
194
res/5/2_th.png
200
res/5/8_th.png
206
res/5/14_th.png
212
res/5/20_th.png
183
res/4/39_th.png
189
res/4/45_th.png
195
res/5/3_th.png
201
res/5/9_th.png
207
res/5/15_th.png
213
res/5/21_th.png
184
res/4/40_th.png
190
res/4/46_th.png
196
res/5/4_th.png
202
res/5/10_th.png
208
res/5/16_th.png
214
res/5/22_th.png
185
res/4/41_th.png
191
res/4/47_th.png
197
res/5/5_th.png
203
res/5/11_th.png
209
res/5/17_th.png
215
res/5/23_th.png


# Sandbox zone below

In [10]:
# Alternative library for SVG parsing

from xml.dom import minidom

doc = minidom.parse("input.svg")
path_strings = [path.getAttribute('d') for path
                in doc.getElementsByTagName('path')]
doc.unlink()

FileNotFoundError: [Errno 2] No such file or directory: 'input.svg'

In [ ]:
# Test .svg draw

img = Image.new("RGBA", (1000,1000)) # last part is image dimensions
draw = aggdraw.Draw(img)
outline = aggdraw.Pen("black", 5) # 5 is the outlinewidth in pixels
fill = aggdraw.Brush("yellow")

# the pathstring:
# m for starting point
# c for bezier curves
# z for closing up the path, optional
# (all lowercase letters for relative path)
pathstring = " c300,300,700,600,300,900 "

# create symbol
symbol = aggdraw.Symbol(pathstring)

xy = (20,20) # xy position to place symbol
draw.symbol(xy, symbol, outline, fill)
draw.flush()

pathstring = " c400,400,800,700,400,1000 "

# create symbol
symbol = aggdraw.Symbol(pathstring)

# draw and save it
xy = (20,20) # xy position to place symbol
draw.symbol(xy, symbol, outline, fill)
draw.flush()
img.save("testbeziercurves.png") # this image gets saved to same folder as the script